In [2]:
# ============================================================
# OBJETIVO: LIMPIEZA DATASETS
# Kyle Field Oil Production Forecasting
# ============================================================

import pandas as pd
import numpy as np
import random
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print("PIPELINE: LIMPIEZA - KYLE FIELD")
print("="*70)

PIPELINE: LIMPIEZA - KYLE FIELD


In [28]:
# ============================================================
# 1. CONFIGURACIÓN Y SEMILLAS
# ============================================================

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    pl.seed_everything(seed, workers=True)

SEED = 42
set_seed(SEED)
print(f"\n[1] Semilla: {SEED}")

Seed set to 42



[1] Semilla: 42


In [ ]:
# ============================================================
# 2. CARGA Y LIMPIEZA DE DATOS
# ============================================================

df1 = pd.read_excel('29_02c- 12z.xlsx')
df2 = pd.read_excel('29_02c- 13.xlsx')
df3= pd.read_excel('29_02c- 14A.xlsx')


print(f"    Well 12: {df1.shape}")
print(f"    Well 13: {df2.shape}")
print(f"    Well 14: {df3.shape}")



    Well 12: (6995, 10)
    Well 13: (6995, 10)
    Well 14: (6788, 10)


In [30]:
# Verificar tipos de datos y valores faltantes en cada dataset
datasets = [df1, df2, df3]
names = ['Dataset_1', 'Dataset_2', 'Dataset_3']

for i, (df, name) in enumerate(zip(datasets, names)):
    print(f"\n=== {name} ===")
    print("Tipos de datos:")
    print(df.dtypes)
    print(f"\nValores faltantes:")
    print(df.isnull().sum())
    print(f"\nRango de fechas:")
    print(f"Inicio: {df['Date'].min()}")
    print(f"Fin: {df['Date'].max()}")

# Agregar identificador de pozo antes de combinar
df1['Well_ID'] = 'Well_12'
df2['Well_ID'] = 'Well_13' 
df3['Well_ID'] = 'Well_14'

df1 = df1.drop(columns=['Platform Choke %'])
df2 = df2.drop(columns=['Platform Choke %'])
df3 = df3.drop(columns=['Platform Choke %'])

print("Columna 'Platform Choke %' eliminada de todos los datasets")


=== Dataset_1 ===
Tipos de datos:
Date                   datetime64[ns]
Hours Online                  float64
Av. WHT (Deg C)               float64
Av. WHP (bar)                 float64
Av. DHT (Deg C)               float64
Av. DHP (bar)                 float64
Platform Choke %              float64
Oil (m3)                      float64
Gas (m3)                      float64
Produced Water (m3)           float64
dtype: object

Valores faltantes:
Date                      0
Hours Online             17
Av. WHT (Deg C)          85
Av. WHP (bar)           890
Av. DHT (Deg C)         199
Av. DHP (bar)           210
Platform Choke %       5089
Oil (m3)                 17
Gas (m3)                 17
Produced Water (m3)      17
dtype: int64

Rango de fechas:
Inicio: 2001-04-08 00:00:00
Fin: 2020-06-01 00:00:00

=== Dataset_2 ===
Tipos de datos:
Date                   datetime64[ns]
Hours Online                  float64
Av. WHT (Deg C)               float64
Av. WHP (bar)                 float64


In [32]:
# ELIMINAR registros sin producción (Oil = 0 AND Gas = 0 AND Water = 0)
print("="*60)
print("ELIMINACION DE REGISTROS SIN PRODUCCIÓN")
print("="*60)

# Well_12
before = len(df1)
df1 = df1[~((df1['Oil (m3)'].isna() | (df1['Oil (m3)'] == 0)) & 
            (df1['Gas (m3)'].isna() | (df1['Gas (m3)'] == 0)) & 
            (df1['Produced Water (m3)'].isna() | (df1['Produced Water (m3)'] == 0)))]
after = len(df1)
print(f"Well_12: {before} → {after} registros (-{before-after})")

# Well_13
before = len(df2)
df2 = df2[~((df2['Oil (m3)'].isna() | (df2['Oil (m3)'] == 0)) & 
            (df2['Gas (m3)'].isna() | (df2['Gas (m3)'] == 0)) & 
            (df2['Produced Water (m3)'].isna() | (df2['Produced Water (m3)'] == 0)))]
after = len(df2)
print(f"Well_13: {before} → {after} registros (-{before-after})")

# Well_14
before = len(df3)
df3 = df3[~((df3['Oil (m3)'].isna() | (df3['Oil (m3)'] == 0)) & 
            (df3['Gas (m3)'].isna() | (df3['Gas (m3)'] == 0)) & 
            (df3['Produced Water (m3)'].isna() | (df3['Produced Water (m3)'] == 0)))]
after = len(df3)
print(f"Well_14: {before} → {after} registros (-{before-after})")

print("\nRegistros sin producción eliminados")

ELIMINACION DE REGISTROS SIN PRODUCCIÓN
Well_12: 4396 → 4396 registros (-0)
Well_13: 3355 → 3355 registros (-0)
Well_14: 4543 → 4543 registros (-0)

Registros sin producción eliminados


In [33]:
# CORRECCIÓN SELECTIVA DE HOURS ONLINE
print("="*60)
print("CORRECCIÓN SELECTIVA DE HOURS ONLINE")
print("="*60)

def selective_correct_hours_online(df, well_name):
    """
    Corrige SOLO Hours Online = 0 cuando hay producción alta
    Mantiene horas pequeñas si la producción también es pequeña
    """
    df_corrected = df.copy()
    
    print(f"\n{well_name}:")
    
    # Calcular tasas normales usando TODO el dataset con Hours > 0
    valid_hours = df_corrected[df_corrected['Hours Online'] > 0].copy()
    
    valid_hours['Oil_rate'] = valid_hours['Oil (m3)'] / valid_hours['Hours Online']
    valid_hours['Gas_rate'] = valid_hours['Gas (m3)'] / valid_hours['Hours Online']
    valid_hours['Water_rate'] = valid_hours['Produced Water (m3)'] / valid_hours['Hours Online']
    
    median_oil_rate = valid_hours['Oil_rate'].median()
    median_gas_rate = valid_hours['Gas_rate'].median()
    median_water_rate = valid_hours['Water_rate'].median()
    
    print(f"  Tasas normales del pozo:")
    print(f"    Oil: {median_oil_rate:.2f} m³/hr")
    print(f"    Gas: {median_gas_rate:.2f} m³/hr")
    print(f"    Water: {median_water_rate:.2f} m³/hr")
    
    # SOLO corregir Hours = 0 cuando hay producción
    zero_hours_mask = (df_corrected['Hours Online'] == 0) & \
                      ((df_corrected['Oil (m3)'] > 0) | 
                       (df_corrected['Gas (m3)'] > 0) | 
                       (df_corrected['Produced Water (m3)'] > 0))
    
    corrections = 0
    
    if zero_hours_mask.sum() > 0:
        print(f"\n  Registros con Hours = 0 pero con producción: {zero_hours_mask.sum()}")
        
        for idx in df_corrected[zero_hours_mask].index:
            oil_prod = df_corrected.loc[idx, 'Oil (m3)']
            gas_prod = df_corrected.loc[idx, 'Gas (m3)']
            water_prod = df_corrected.loc[idx, 'Produced Water (m3)']
            
            estimated_hours = []
            
            if oil_prod > 0 and median_oil_rate > 0:
                estimated_hours.append(oil_prod / median_oil_rate)
            
            if gas_prod > 0 and median_gas_rate > 0:
                estimated_hours.append(gas_prod / median_gas_rate)
            
            if water_prod > 0 and median_water_rate > 0:
                estimated_hours.append(water_prod / median_water_rate)
            
            if estimated_hours:
                estimated = np.median(estimated_hours)
                estimated = min(estimated, 24)
                estimated = max(estimated, 0.1)
                
                df_corrected.loc[idx, 'Hours Online'] = estimated
                corrections += 1
    
    # Corregir Hours > 24 (físicamente imposible)
    over_24_mask = df_corrected['Hours Online'] > 24
    if over_24_mask.sum() > 0:
        print(f"  Registros con Hours > 24: {over_24_mask.sum()}")
        df_corrected.loc[over_24_mask, 'Hours Online'] = 24
        corrections += over_24_mask.sum()
    
    print(f"\n  Total correcciones: {corrections}")
    
    return df_corrected

df1 = selective_correct_hours_online(df1, 'Well_12')
df2 = selective_correct_hours_online(df2, 'Well_13')
df3 = selective_correct_hours_online(df3, 'Well_14')

CORRECCIÓN SELECTIVA DE HOURS ONLINE

Well_12:
  Tasas normales del pozo:
    Oil: 7.57 m³/hr
    Gas: 6724.60 m³/hr
    Water: 1.00 m³/hr

  Registros con Hours = 0 pero con producción: 53
  Registros con Hours > 24: 3

  Total correcciones: 56

Well_13:
  Tasas normales del pozo:
    Oil: 9.86 m³/hr
    Gas: 10407.21 m³/hr
    Water: 9.51 m³/hr

  Registros con Hours = 0 pero con producción: 24
  Registros con Hours > 24: 2

  Total correcciones: 26

Well_14:
  Tasas normales del pozo:
    Oil: 20.26 m³/hr
    Gas: 2321.39 m³/hr
    Water: 0.00 m³/hr

  Registros con Hours = 0 pero con producción: 24
  Registros con Hours > 24: 1

  Total correcciones: 21


In [35]:
print("\n" + "="*60)
print("CORRECCIÓN SELECTIVA COMPLETADA")
print("="*60)

# CORRECCIÓN DE Av. WHT (Deg C)
print("="*60)
print("CORRECCIÓN DE Av. WHT (Deg C)")
print("="*60)

def correct_wht_temperature(df, well_name):
    """Corrige temperaturas Av. WHT (Deg C) con problemas"""
    df_corrected = df.copy()
    corrections = {}
    
    print(f"\n{well_name}:")
    print(f"  Total registros: {len(df_corrected)}")
    
    valid_temp = df_corrected['Av. WHT (Deg C)']
    valid_temp = valid_temp[(valid_temp > 0) & (valid_temp < 150)]
    median_temp = valid_temp.median()
    
    print(f"  Temperatura mediana válida: {median_temp:.2f}°C")
    
    # Temperaturas negativas
    neg_mask = df_corrected['Av. WHT (Deg C)'] < 0
    if neg_mask.sum() > 0:
        print(f"\n  Corrigiendo T < 0°C: {neg_mask.sum()} registros")
        df_corrected.loc[neg_mask, 'Av. WHT (Deg C)'] = median_temp
        corrections['negative'] = neg_mask.sum()
    
    # Temperaturas > 150°C
    high_mask = df_corrected['Av. WHT (Deg C)'] > 150
    if high_mask.sum() > 0:
        print(f"  Corrigiendo T > 150°C: {high_mask.sum()} registros")
        df_corrected.loc[high_mask, 'Av. WHT (Deg C)'] = median_temp
        corrections['high'] = high_mask.sum()
    
    # T = 0 con producción
    zero_with_prod = (df_corrected['Av. WHT (Deg C)'] == 0) & \
                     ((df_corrected['Oil (m3)'] > 0) | (df_corrected['Gas (m3)'] > 0))
    
    if zero_with_prod.sum() > 0:
        print(f"  Corrigiendo T=0 con producción: {zero_with_prod.sum()} registros")
        df_corrected.loc[zero_with_prod, 'Av. WHT (Deg C)'] = median_temp
        corrections['zero_with_prod'] = zero_with_prod.sum()
    
    # Valores nulos
    null_mask = df_corrected['Av. WHT (Deg C)'].isnull()
    if null_mask.sum() > 0:
        print(f"  Imputando valores nulos: {null_mask.sum()} registros")
        df_corrected.loc[null_mask, 'Av. WHT (Deg C)'] = median_temp
        corrections['nulls'] = null_mask.sum()
    
    # Mantener T=0 sin producción
    zero_no_prod = (df_corrected['Av. WHT (Deg C)'] == 0) & \
                   (df_corrected['Oil (m3)'] == 0) & \
                   (df_corrected['Gas (m3)'] == 0)
    print(f"  Manteniendo T=0 sin producción (válido): {zero_no_prod.sum()} registros")
    
    total_corrections = sum(corrections.values())
    print(f"\n  Total correcciones: {total_corrections}")
    
    return df_corrected

df1 = correct_wht_temperature(df1, 'Well_12')
df2 = correct_wht_temperature(df2, 'Well_13')
df3 = correct_wht_temperature(df3, 'Well_14')




CORRECCIÓN SELECTIVA COMPLETADA
CORRECCIÓN DE Av. WHT (Deg C)

Well_12:
  Total registros: 4396
  Temperatura mediana válida: 68.00°C
  Manteniendo T=0 sin producción (válido): 3 registros

  Total correcciones: 0

Well_13:
  Total registros: 3355
  Temperatura mediana válida: 69.44°C
  Manteniendo T=0 sin producción (válido): 0 registros

  Total correcciones: 0

Well_14:
  Total registros: 4543
  Temperatura mediana válida: 56.00°C
  Manteniendo T=0 sin producción (válido): 0 registros

  Total correcciones: 0


In [36]:
# CORRECCIÓN DE Av. WHP (bar)
print("="*60)
print("CORRECCIÓN DE Av. WHP (bar)")
print("="*60)

def correct_whp_pressure(df, well_name):
    """Corrige presiones Av. WHP (bar) con problemas"""
    df_corrected = df.copy()
    corrections = {}
    
    print(f"\n{well_name}:")
    print(f"  Total registros: {len(df_corrected)}")
    
    valid_pressure = df_corrected['Av. WHP (bar)']
    valid_pressure = valid_pressure[valid_pressure > 0]
    median_pressure = valid_pressure.median()
    
    print(f"  Presión mediana válida del pozo: {median_pressure:.2f} bar")
    
    # Valores = 0 con producción
    zero_with_prod = (df_corrected['Av. WHP (bar)'] == 0) & \
                     ((df_corrected['Oil (m3)'] > 0) | 
                      (df_corrected['Gas (m3)'] > 0))
    
    if zero_with_prod.sum() > 0:
        print(f"\n  Corrigiendo P=0 con producción: {zero_with_prod.sum()} registros")
        df_corrected.loc[zero_with_prod, 'Av. WHP (bar)'] = median_pressure
        corrections['zero_with_prod'] = zero_with_prod.sum()
    
    # Valores nulos
    null_mask = df_corrected['Av. WHP (bar)'].isnull()
    if null_mask.sum() > 0:
        print(f"  Imputando valores nulos: {null_mask.sum()} registros")
        df_corrected.loc[null_mask, 'Av. WHP (bar)'] = median_pressure
        corrections['nulls'] = null_mask.sum()
    
    # Outliers extremos > 500 bar
    extreme_mask = df_corrected['Av. WHP (bar)'] > 500
    if extreme_mask.sum() > 0:
        print(f"  Corrigiendo P > 500 bar: {extreme_mask.sum()} registros")
        df_corrected.loc[extreme_mask, 'Av. WHP (bar)'] = median_pressure
        corrections['extreme'] = extreme_mask.sum()
    
    # Mantener P=0 sin producción
    zero_no_prod = (df_corrected['Av. WHP (bar)'] == 0) & \
                   (df_corrected['Oil (m3)'] == 0) & \
                   (df_corrected['Gas (m3)'] == 0)
    print(f"  Manteniendo P=0 sin producción (válido): {zero_no_prod.sum()} registros")
    
    total_corrections = sum(corrections.values())
    print(f"\n  Total correcciones: {total_corrections}")
    
    return df_corrected

df1 = correct_whp_pressure(df1, 'Well_12')
df2 = correct_whp_pressure(df2, 'Well_13')
df3 = correct_whp_pressure(df3, 'Well_14')

CORRECCIÓN DE Av. WHP (bar)

Well_12:
  Total registros: 4396
  Presión mediana válida del pozo: 65.98 bar

  Corrigiendo P=0 con producción: 934 registros
  Imputando valores nulos: 515 registros
  Manteniendo P=0 sin producción (válido): 0 registros

  Total correcciones: 1449

Well_13:
  Total registros: 3355
  Presión mediana válida del pozo: 51.99 bar

  Corrigiendo P=0 con producción: 26 registros
  Imputando valores nulos: 52 registros
  Manteniendo P=0 sin producción (válido): 0 registros

  Total correcciones: 78

Well_14:
  Total registros: 4543
  Presión mediana válida del pozo: 36.99 bar

  Corrigiendo P=0 con producción: 710 registros
  Imputando valores nulos: 445 registros
  Manteniendo P=0 sin producción (válido): 0 registros

  Total correcciones: 1155


In [37]:
# CORRECCIÓN DE DHT Y DHP
print("="*60)
print("CORRECCIÓN DE DHT Y DHP")
print("="*60)

def correct_dht_dhp(df, well_name):
    """Corrige temperaturas DHT y presiones DHP"""
    df_corrected = df.copy()
    corrections = {}
    
    print(f"\n{well_name}:")
    
    # CORRECCIÓN DHT
    valid_dht = df_corrected['Av. DHT (Deg C)']
    valid_dht = valid_dht[(valid_dht > 0) & (valid_dht < 150)]
    median_dht = valid_dht.median()
    print(f"  DHT mediana válida: {median_dht:.2f}°C")
    
    neg_dht = df_corrected['Av. DHT (Deg C)'] < 0
    if neg_dht.sum() > 0:
        df_corrected.loc[neg_dht, 'Av. DHT (Deg C)'] = median_dht
        corrections['dht_negative'] = neg_dht.sum()
    
    high_dht = df_corrected['Av. DHT (Deg C)'] > 150
    if high_dht.sum() > 0:
        df_corrected.loc[high_dht, 'Av. DHT (Deg C)'] = median_dht
        corrections['dht_high'] = high_dht.sum()
    
    zero_dht_prod = (df_corrected['Av. DHT (Deg C)'] == 0) & \
                    ((df_corrected['Oil (m3)'] > 0) | (df_corrected['Gas (m3)'] > 0))
    if zero_dht_prod.sum() > 0:
        df_corrected.loc[zero_dht_prod, 'Av. DHT (Deg C)'] = median_dht
        corrections['dht_zero_prod'] = zero_dht_prod.sum()
    
    null_dht = df_corrected['Av. DHT (Deg C)'].isnull()
    if null_dht.sum() > 0:
        df_corrected.loc[null_dht, 'Av. DHT (Deg C)'] = median_dht
        corrections['dht_nulls'] = null_dht.sum()
    
    # CORRECCIÓN DHP
    valid_dhp = df_corrected['Av. DHP (bar)']
    valid_dhp = valid_dhp[valid_dhp > 0]
    median_dhp = valid_dhp.median()
    print(f"  DHP mediana válida: {median_dhp:.2f} bar")
    
    neg_dhp = df_corrected['Av. DHP (bar)'] < 0
    if neg_dhp.sum() > 0:
        df_corrected.loc[neg_dhp, 'Av. DHP (bar)'] = median_dhp
        corrections['dhp_negative'] = neg_dhp.sum()
    
    high_dhp = df_corrected['Av. DHP (bar)'] > 500
    if high_dhp.sum() > 0:
        df_corrected.loc[high_dhp, 'Av. DHP (bar)'] = median_dhp
        corrections['dhp_high'] = high_dhp.sum()
    
    zero_dhp_prod = (df_corrected['Av. DHP (bar)'] == 0) & \
                    ((df_corrected['Oil (m3)'] > 0) | (df_corrected['Gas (m3)'] > 0))
    if zero_dhp_prod.sum() > 0:
        df_corrected.loc[zero_dhp_prod, 'Av. DHP (bar)'] = median_dhp
        corrections['dhp_zero_prod'] = zero_dhp_prod.sum()
    
    null_dhp = df_corrected['Av. DHP (bar)'].isnull()
    if null_dhp.sum() > 0:
        df_corrected.loc[null_dhp, 'Av. DHP (bar)'] = median_dhp
        corrections['dhp_nulls'] = null_dhp.sum()
    
    total = sum(corrections.values())
    print(f"  Total correcciones DHT+DHP: {total}")
    
    return df_corrected

df1 = correct_dht_dhp(df1, 'Well_12')
df2 = correct_dht_dhp(df2, 'Well_13')
df3 = correct_dht_dhp(df3, 'Well_14')

CORRECCIÓN DE DHT Y DHP

Well_12:
  DHT mediana válida: 94.00°C
  DHP mediana válida: 91.56 bar
  Total correcciones DHT+DHP: 1768

Well_13:
  DHT mediana válida: 95.50°C
  DHP mediana válida: 105.14 bar
  Total correcciones DHT+DHP: 396

Well_14:
  DHT mediana válida: 99.00°C
  DHP mediana válida: 97.91 bar
  Total correcciones DHT+DHP: 1058


In [38]:
# UNIR LOS 3 DATASETS LIMPIOS
print("="*60)
print("UNIENDO DATASETS LIMPIOS")
print("="*60)

print("\nRegistros por pozo ANTES de unir:")
print(f"  Well_12: {len(df1)} registros")
print(f"  Well_13: {len(df2)} registros")
print(f"  Well_14: {len(df3)} registros")
print(f"  TOTAL: {len(df1) + len(df2) + len(df3)} registros")

df_combined = pd.concat([df1, df2, df3], ignore_index=True)

print(f"\nDatasets unidos exitosamente")
print(f"Dataset combinado: {df_combined.shape}")

print(f"\nDistribución por pozo:")
print(df_combined['Well_ID'].value_counts().sort_index())

coordinates = {
    'Well_12': {'X': 1.23349, 'Y': 56.8859},
    'Well_13': {'X': 1.22352, 'Y': 56.9148},
    'Well_14': {'X': 1.22292, 'Y': 56.9146},
}

UNIENDO DATASETS LIMPIOS

Registros por pozo ANTES de unir:
  Well_12: 4396 registros
  Well_13: 3355 registros
  Well_14: 4543 registros
  TOTAL: 12294 registros

Datasets unidos exitosamente
Dataset combinado: (12294, 10)

Distribución por pozo:
Well_ID
Well_12    4396
Well_13    3355
Well_14    4543
Name: count, dtype: int64


In [39]:
# Agregar coordenadas al dataset
df_combined['X'] = df_combined['Well_ID'].map(lambda well_id: coordinates[well_id]['X'])
df_combined['Y'] = df_combined['Well_ID'].map(lambda well_id: coordinates[well_id]['Y'])

print("\nDataset con coordenadas:")
print(df_combined[['Well_ID', 'X', 'Y']].drop_duplicates())

# Calcular distancias entre pozos
well_coords = df_combined[['Well_ID', 'X', 'Y']].drop_duplicates().set_index('Well_ID')
coords_array = well_coords[['X', 'Y']].values

distances = pdist(coords_array, metric='euclidean')
distance_matrix = squareform(distances)

print(f"\nDistancias entre pozos (en grados):")
print(pd.DataFrame(distance_matrix, 
                   index=well_coords.index, 
                   columns=well_coords.index).round(4))


Dataset con coordenadas:
      Well_ID        X        Y
0     Well_12  1.23349  56.8859
4396  Well_13  1.22352  56.9148
7751  Well_14  1.22292  56.9146

Distancias entre pozos (en grados):
Well_ID  Well_12  Well_13  Well_14
Well_ID                           
Well_12   0.0000   0.0306   0.0306
Well_13   0.0306   0.0000   0.0006
Well_14   0.0306   0.0006   0.0000


In [40]:
# DIVISIÓN ESTRATIFICADA
print("\n" + "="*70)
print("DIVISIÓN ESTRATIFICADA POR POZO")
print("="*70)

def split_stratified(df_combined, train_ratio=0.70, val_ratio=0.15):
    """Split cada pozo independientemente"""
    train_list, val_list, test_list = [], [], []
    
    print("\nDivisión por pozo (70% / 15% / 15%):\n")
    
    for well in sorted(df_combined['Well_ID'].unique()):
        df_well = df_combined[df_combined['Well_ID'] == well].sort_values('Date')
        dates = sorted(df_well['Date'].unique())
        n = len(dates)
        
        t_idx = int(train_ratio * n)
        v_idx = int((train_ratio + val_ratio) * n)
        
        train_end = dates[t_idx]
        val_end = dates[v_idx]
        
        train_well = df_well[df_well['Date'] <= train_end]
        val_well = df_well[(df_well['Date'] > train_end) & (df_well['Date'] <= val_end)]
        test_well = df_well[df_well['Date'] > val_end]
        
        print(f"{well}:")
        print(f"  Train: {dates[0].strftime('%Y-%m-%d')} → {train_end.strftime('%Y-%m-%d')} ({len(train_well):4,} registros, {len(train_well)/len(df_well)*100:.1f}%)")
        print(f"  Val:   {dates[t_idx+1].strftime('%Y-%m-%d')} → {val_end.strftime('%Y-%m-%d')} ({len(val_well):4,} registros, {len(val_well)/len(df_well)*100:.1f}%)")
        print(f"  Test:  {dates[v_idx+1].strftime('%Y-%m-%d')} → {dates[-1].strftime('%Y-%m-%d')} ({len(test_well):4,} registros, {len(test_well)/len(df_well)*100:.1f}%)")
        print()
        
        train_list.append(train_well)
        val_list.append(val_well)
        test_list.append(test_well)
    
    return (pd.concat(train_list).sort_values(['Date', 'Well_ID']),
            pd.concat(val_list).sort_values(['Date', 'Well_ID']),
            pd.concat(test_list).sort_values(['Date', 'Well_ID']))

train_strat, val_strat, test_strat = split_stratified(df_combined)

print("TOTALES CON DIVISIÓN ESTRATIFICADA:")
print(f"  Train: {len(train_strat):5,} registros ({len(train_strat)/len(df_combined)*100:.1f}%)")
print(f"  Val:   {len(val_strat):5,} registros ({len(val_strat)/len(df_combined)*100:.1f}%)")
print(f"  Test:  {len(test_strat):5,} registros ({len(test_strat)/len(df_combined)*100:.1f}%)")

print("\nVerificación de pozos:")
print(f"  Train: {sorted(train_strat['Well_ID'].unique())}")
print(f"  Val:   {sorted(val_strat['Well_ID'].unique())}")
print(f"  Test:  {sorted(test_strat['Well_ID'].unique())}")

if set(test_strat['Well_ID'].unique()) == {'Well_12', 'Well_13', 'Well_14'}:
    print("\n  TODOS los pozos presentes en test!")


DIVISIÓN ESTRATIFICADA POR POZO

División por pozo (70% / 15% / 15%):

Well_12:
  Train: 2001-04-08 → 2011-10-26 (3,078 registros, 70.0%)
  Val:   2011-10-27 → 2018-04-19 ( 659 registros, 15.0%)
  Test:  2018-04-20 → 2020-06-01 ( 659 registros, 15.0%)

Well_13:
  Train: 2001-04-09 → 2016-07-16 (2,349 registros, 70.0%)
  Val:   2016-07-17 → 2018-11-09 ( 503 registros, 15.0%)
  Test:  2018-11-10 → 2020-06-01 ( 503 registros, 15.0%)

Well_14:
  Train: 2001-11-11 → 2011-09-12 (3,181 registros, 70.0%)
  Val:   2011-09-13 → 2016-05-13 ( 681 registros, 15.0%)
  Test:  2016-05-14 → 2018-10-09 ( 681 registros, 15.0%)

TOTALES CON DIVISIÓN ESTRATIFICADA:
  Train: 8,608 registros (70.0%)
  Val:   1,843 registros (15.0%)
  Test:  1,843 registros (15.0%)

Verificación de pozos:
  Train: ['Well_12', 'Well_13', 'Well_14']
  Val:   ['Well_12', 'Well_13', 'Well_14']
  Test:  ['Well_12', 'Well_13', 'Well_14']

  TODOS los pozos presentes en test!


In [ ]:
# ============================================================
# GUARDAR DATASETS LIMPIOS
# ============================================================

train_strat.to_csv('train_estratificado.csv', index=False)
val_strat.to_csv('val_estratificado.csv', index=False)
test_strat.to_csv('test_estratificado.csv', index=False)

print("\nDatasets guardados:")
print("   - train_estratificado.csv")
print("   - val_estratificado.csv")
print("   - test_estratificado.csv")

print("\n" + "="*70)
print("LIMPIEZA COMPLETADA")
print("="*70)